In [ ]:
#test using regressions with pandas
import pandas as pd
import numpy as np
import statsmodels.api as sm
import time

/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
#test using regressions with pandas
import pandas as pd
import numpy as np
import statsmodels.api as sm
import time

#function to write instrumental varibles into their respective data sets
def IVReg(dependent, endog, exog, instrument):
	#get results of first stage regression
	name = endog.name		#name of the column with the FDI data. it's stored here so we can add it to the transformed data set after the first stage regression
	#add instrument to exog
	instrument = instrument.to_frame()
	exogfirst = pd.merge(exog, instrument, left_index = True, right_index = True)
	exogfirst = sm.add_constant(exogfirst)
	#run first stage regression
	results = sm.OLS(endog, exogfirst, missing = 'drop').fit()
	fittedResults = results.fittedvalues
	
	#the first regression dropped rows with missing values, so we need to input NaN in them again
	last = -1
	for k, v in fittedResults.iteritems():
		if k > last + 1:
			for i in range(last+1, k):
				fittedResults = fittedResults.set_value(i, np.NaN)
		last = k
	
	#in case there are missing values at the end
	while fittedResults.shape[0] < exog.shape[0]:
		last += 1
		fittedResults = fittedResults.set_value(last, np.NaN)
	
	#sort results, change to a data frame, and give it a name
	fittedResults = fittedResults.sort_index()
	fittedResults = fittedResults.to_frame('fitted ' + name)
	
	#for debugging
	exog.to_csv('debugSheetIV.csv')
	
	#include the results from the first stage in the matrix for the second stage regression
	exog = pd.merge(fittedResults, exog, left_index = True, right_index = True)
	
	#add constant
	exog = sm.add_constant(exog)
	
	#return the results from the second stage	
	finalResults = sm.OLS(dependent, exog, missing = 'drop').fit()
	
	return finalResults

#run program
if __name__ == "__main__":
	
	#read data into frames
	indk = pd.read_csv('Industry_keys.csv')
	devindk = pd.read_csv('Development_industry_keys.csv')
	
	#initialize arrays to store results
	jobsarr = []
	capexarr = []
	jobsallarr = []
	capexallarr = []
	jobsmanarr = []
	capexmanarr = []
	indicators = []

	#determine control variables
	potentialControls = ['GDP per capita t-1', 'Trade openness t-1', 'Gross domestic savings (% of GDP)', 'War dummy', 'Net official development assistance and official aid received (constant 2013 US$)', 'External debt stocks (% of GNI)', 'Share of FDI in manufacturing', 'Share of FDI in extraction', 'Share of FDI in high value-added industries', 'Individuals using the Internet (% of population)', 'Arms imports (SIPRI trend indicator values)', 'Armed forces personnel (% of total labor force)', 'Chinn-Ito financial openness index']
	controls = []
	
	#determine lag for dependent variable
	lagAnswer = 0
	
	#menu for lag
	print('\nWould you like to add a lag to your dependent variables?\n')
	print('0: No lag')
	print('1: One year lag')
	print('2: Two years lag')
	print('3: Three years lag')
	#ask for input
	lagAnswer = input('\nSelect 0 - 3 to determine your lag structure: ')
	#check if valid input
	try:
		lagAnswer = int(lagAnswer)
	except ValueError:
		print('\nNot a valid input. Default is no lag.\n')
		lagAnswer = 0
	#check if answer was out of bound
	if lagAnswer < 0 or lagAnswer > 3:
		print('\nNot a valid input. Default is no lag.\n')
		lagAnswer = 0
	
	#load dataset based on lag input
	if lagAnswer == 0:
		capex = pd.read_csv('Capex.csv')
		jobs = pd.read_csv('Jobs.csv')
	else:
		capexName = 'Capex' + str(lagAnswer) + '.csv'
		jobsName = 'Jobs' + str(lagAnswer) + '.csv'
		capex = pd.read_csv(capexName)
		jobs = pd.read_csv(jobsName)
	
	#identify missing values in data sets
	capex = capex.replace(-1, np.NaN)
	jobs = jobs.replace(-1, np.NaN)
	
	#used to control the menu for control variables
	controlCheck = 0
	
	#menu for control variables
	print('\nWould you like to add control variables?\n')
	while controlCheck == 0:	
		#display options for control variables
		print('\n0 :  No additional controls')
		for i in range(1, len(potentialControls)+1):
			print(i, ': ', potentialControls[i-1])
		#ask which control variables the user wants to include
		controlString = '\nSelect controls 1 - ' + str(len(potentialControls)) + ' to add to your regression, or select 0 to leave it be: '
		controlAnswer = input(controlString)
		#check if valid response
		try:
			controlAnswer = int(controlAnswer)
		except ValueError:
			print('\nNot a valid input. Default is no control variables.\n')
			continue
		#add control variable, ask again, or leave
		if controlAnswer == 0:
			controlCheck = 1
		elif 0 < controlAnswer < len(potentialControls)+1:
			controls.append(potentialControls[controlAnswer-1])
			del potentialControls[controlAnswer-1]
	
	#determine fixed effects control
	fixedTimeCheck = 0
	fixedCountryCheck = 0
	
	#menu for fixed effects
	print('\nWould you like to control for fixed effects?\n')
	print('0 : No fixed effects controls')
	print('1 : Control for time fixed effects')
	print('2 : Control for country fixed effects')
	print('3 : Control for both fixed effects')
	fixedAnswer = input('\nSelect 0 - 3 to determine fixed effects controls: ')
	#check if valid response
	try:
		fixedAnswer = int(fixedAnswer)
	except ValueError:
		print('\nNot a valid input. default is no fixed effects controls.\n')
	#set fixed effects parameter based on input
	if fixedAnswer == 1:
		fixedTimeCheck = 1
	elif fixedAnswer == 2:
		fixedCountryCheck = 1
	elif fixedAnswer == 3:
		fixedTimeCheck = 1
		fixedCountryCheck = 1
		
	#determine instrumental variable
	instrumentCheck = 0
	instrumentVector = ['Imports of goods and services (constant 2010 US$)', 'Chinn-Ito financial openness index']
	
	#menu for instrumental variable estimation
	print('\nWould you like to employ a 2-stage least squares instrumental variable estimation?\n')
	print('0:  No instrumental variable')
	print('1: ', instrumentVector[0])
	print('2: ', instrumentVector[1])
	instrumentAnswer = input('\nSelect 0 - 2 to determine instrumental variable estimation: ')
	#check if valid response
	try:
		instrumentAnswer = int(instrumentAnswer)
	except ValueError:
		print('\nNot a valid input. Default is no instrumental variable.\n')
	#set instrumental variable parameters based on input
	if instrumentAnswer == 1:
		instrument = jobs[instrumentVector[0]]
		instrumentCheck = 1
	elif instrumentAnswer == 2:
		instrument = jobs[instrumentVector[1]]
		instrumentCheck = 1
	
	#determine interaction effects
	interactionVector = ['Political Participation (EIU/FH/V-Dem)', 'IIAG governance index', 'Tertiary Education Enrolment (UNESCO)', 'Access to electricity (% of population)', 'Mobile phone subscriptions per 100 people']
	
	#menu for interaction effects
	print('\nWould you like to look for interaction effects?\n')
	print('0:  No interaction effects')
	print('1: ', interactionVector[0])
	print('2: ', interactionVector[1])
	print('3: ', interactionVector[2])
	print('4: ', interactionVector[3])
	print('5: ', interactionVector[4])
	interactionAnswer = input('\nSelect 0 - 5 to determine interaction effects: ')
	#check if valid response
	try:
		interactionAnswer = int(interactionAnswer)
	except ValueError:
		print('\nNot a valid input. Default is no interaction effects.\n')
	#set interaction effects
	interaction = 0				#this tracks whether we're using interaction effects, and what it is
	if interactionAnswer == 1:
		interaction = interactionVector[0]
		controls.append(interaction)
	elif interactionAnswer == 2:
		interaction = interactionVector[1]
		controls.append(interaction)
	elif interactionAnswer == 3:
		interaction = interactionVector[2]
		controls.append(interaction)
	elif interactionAnswer == 4:
		interaction = interactionVector[3]
		controls.append(interaction)
	elif interactionAnswer == 5:
		interaction = interactionVector[4]
		controls.append(interaction)
	#generate data series for non-industry specific interaction terms
	if interaction != 0:
		jobsallInter = jobs['allall']
		jobsallInter = jobsallInter.multiply(capex[interaction])
		jobsallInter = jobsallInter.to_frame(interaction + ' inter')
		capexallInter = capex['allall']
		capexallInter = capexallInter.multiply(capex[interaction])
		capexallInter = capexallInter.to_frame(interaction + ' inter')
		jobsmanInter = jobs['allman']
		jobsmanInter = jobsmanInter.multiply(capex[interaction])
		jobsmanInter = jobsmanInter.to_frame(interaction + ' inter')
		capexmanInter = capex['allman']
		capexmanInter = capexmanInter.multiply(capex[interaction])
		capexmanInter = capexmanInter.to_frame(interaction + ' inter')
	
	#time the program
	start = time.time()
	
	#message to let user know the work takes a little time
	print('\nLOADING...\n')
	
	#store vector of countries, excluding Zimbabwe
	countries = ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 'Congo (DRC)', "Cote d'Ivoire", 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Republic of the Congo', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Swaziland', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia']
	
	#store vector of years, excluding 2016
	years = ['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
	
	#include fixed effects by country
	if fixedCountryCheck == 1:
		controls.extend(countries)
	#include fixed effects by year
	if fixedTimeCheck == 1:
		#make sure we store the right fixed effects controls for our lag structure
		if lagAnswer >= 1:
			del years[-1]
		if lagAnswer >= 2:
			del years[-1]
		if lagAnswer >= 3:
			del years[-1]
		controls.extend(years)
	
	#initialize and store regressor vecotrs
	x = []									#this is stored later, since it changes with industries
	xall = ['allall']
	xman = ['allman']
	x.extend(controls)
	xall.extend(controls)
	xman.extend(controls)
	xall1 = jobs[xall]						#store columns for all jobs created by FDI with controls
	xall2 = capex[xall]						#store columns for total millions in FDI invested with controls
	xman1 = jobs[xman]						#store columns for jobs created by FDI for the manufacturing activity with controls
	xman2 = capex[xman]						#store columns for millions in FDI invested for the manufacturing activity with controls
	
	#include interaction effects if we have them
	if interaction != 0:
		xall1 = pd.merge(xall1, jobsallInter, left_index = True, right_index = True)
		xall2 = pd.merge(xall2, capexallInter, left_index = True, right_index = True)
		xman1 = pd.merge(xman1, jobsmanInter, left_index = True, right_index = True)
		xman2 = pd.merge(xman2, capexmanInter, left_index = True, right_index = True)
		
	#FOR DEBUGGING
	xall1.to_csv('debugSheet.csv')
	
	#loop through development indicators and find the industries to which they are related
	for i, row in devindk.iterrows():
		industry = row[2]										#store the industry relevant to the development indicator
		devName = row[0]							#store the name for the development indicator
		#loop through industries to find the code for the relevant industry to the development indicator
		for i, row in indk.iterrows():
			if row[0] == industry and devName != interaction:								#we've arrived at the industry that matches the development indicator
				FDIcode = row[1] + 'all'						#store the industry code
				#store the name of the development indicator
				if lagAnswer != 0:
					#remove previous concurrent development indicator
					if len(indicators) > 0 and instrumentCheck == 0:
						xall1 = xall1.drop(curDevName, 1)
						xall2 = xall2.drop(curDevName, 1)
						xman1 = xman1.drop(curDevName, 1)
						xman2 = xman2.drop(curDevName, 1)
					#store name of lagged and concurrent development indicator
					curDevName = devName
					devName = devName + str(lagAnswer)
					#turn concurrent development indicator into data frame and merge into regressor matrices
					if instrumentCheck == 0:
						jobsCurDev = jobs[curDevName]
						capexCurDev = capex[curDevName]
						jobsCurDev = jobsCurDev.to_frame(curDevName)
						capexCurDev = capexCurDev.to_frame(curDevName)
						xall1 = pd.merge(xall1, jobsCurDev, left_index = True, right_index = True)
						xall2 = pd.merge(xall2, capexCurDev, left_index = True, right_index = True)
						xman1 = pd.merge(xman1, jobsCurDev, left_index = True, right_index = True)
						xman2 = pd.merge(xman2, capexCurDev, left_index = True, right_index = True)
						x.insert(0, curDevName)
					
					#add concurrent development to exog/controls
					controls.insert(0, curDevName)
					indicators.append(curDevName)
				else:
					indicators.append(devName)
				#store column for development indicator
				y = capex[devName]
				#generate interaction terms for industry-specific FDI
				if interaction != 0:
					jobsInter = jobs[FDIcode]
					jobsInter = jobsInter.multiply(capex[interaction])
					jobsInter = jobsInter.to_frame(interaction + ' inter')
					capexInter = capex[FDIcode]
					capexInter = capexInter.multiply(capex[interaction])
					capexInter = capexInter.to_frame(interaction + ' inter')
				#check if we're doing instrumental variable regressions, or regular OLS
				if instrumentCheck == 0:
					#add independent variables to regressor vector
					x.insert(0, FDIcode)
					#store full industry-specific regressor matrices
					x1 = jobs[x]								#store columns for jobs created by FDI in that industry with controls
					x2 = capex[x]								#store columns for millions in FDI invested in that industry with controls
					#include interaction effects if we have them
					if interaction != 0:
						x1 = pd.merge(x1, jobsInter, left_index = True, right_index = True)
						x2 = pd.merge(x2, capexInter, left_index = True, right_index = True)
					#for debugging
					x1.to_csv('debugSheet.csv')
					#add constants to datasets
					x1 = sm.add_constant(x1)
					x2 = sm.add_constant(x2)
					xall1 = sm.add_constant(xall1)
					xall2 = sm.add_constant(xall2)
					xman1 = sm.add_constant(xman1)
					xman2 = sm.add_constant(xman2)
					#regress development indicator against regressors, and store the results in arrays
					jobsarr.append(sm.OLS(y, x1, missing = 'drop').fit())
					capexarr.append(sm.OLS(y, x2, missing = 'drop').fit())
					jobsallarr.append(sm.OLS(y, xall1, missing = 'drop').fit())
					capexallarr.append(sm.OLS(y, xall2, missing = 'drop').fit())
					jobsmanarr.append(sm.OLS(y, xman1, missing = 'drop').fit())
					capexmanarr.append(sm.OLS(y, xman2, missing = 'drop').fit())
					#remove independent variable from the industry-specific regressor vector, so we can restart with a new industry
					x = x[1:]
					#remove concurrent development indicator if we're uding lags
					if lagAnswer != 0:
						x = x[1:]
					x1 = []
					x2 = []
				else:
					#declare endogenous variables
					endog1 = jobs[FDIcode]
					endog2 = capex[FDIcode]
					endogall1 = jobs['allall']
					endogall2 = capex['allall']
					endogman1 = jobs['allman']
					endogman2 = capex['allman']
					#declare exogenous variables
					exog = jobs[controls]
					#include interaction effects if we have them
					if interaction != 0:
						#generate new regressor matrices including the interaction terms
						exog1 = pd.merge(exog, jobsInter, left_index = True, right_index = True)
						exog2 = pd.merge(exog, capexInter, left_index = True, right_index = True)
						exogall1 = pd.merge(exog, jobsallInter, left_index = True, right_index = True)
						exogall2 = pd.merge(exog, capexallInter, left_index = True, right_index = True)
						exogman1 = pd.merge(exog, jobsmanInter, left_index = True, right_index = True)
						exogman2 = pd.merge(exog, capexmanInter, left_index = True, right_index = True)
						#for debugging
						exog1.to_csv('debugSheetInter.csv')
						#add constants to data sets
						exog1 = sm.add_constant(exog1)
						exog2 = sm.add_constant(exog2)
						exogall1 = sm.add_constant(exogall1)
						exogall2 = sm.add_constant(exogall2)
						exogman1 = sm.add_constant(exogman1)
						exogman2 = sm.add_constant(exogman2)
						#run regressions
						jobsarr.append(IVReg(y, endog1, exog1, instrument))
						capexarr.append(IVReg(y, endog2, exog2, instrument))
						jobsallarr.append(IVReg(y, endogall1, exogall1, instrument))
						capexallarr.append(IVReg(y, endogall2, exogall2, instrument))
						jobsmanarr.append(IVReg(y, endogman1, exogman1, instrument))
						capexmanarr.append(IVReg(y, endogman2, exogman2, instrument))
					#run instrumental variable regressions and store results in arrays
					else:
						jobsarr.append(IVReg(y, endog1, exog, instrument))
						capexarr.append(IVReg(y, endog2, exog, instrument))
						jobsallarr.append(IVReg(y, endogall1, exog, instrument))
						capexallarr.append(IVReg(y, endogall2, exog, instrument))
						jobsmanarr.append(IVReg(y, endogman1, exog, instrument))
						capexmanarr.append(IVReg(y, endogman2, exog, instrument))
					#remove concurrent development indicator if we're using lag
					if lagAnswer != 0:
						controls = controls[1:]
				break
	
	#finish timing program here and print
	end = time.time()
	elapsed = int(end - start)
	print('\nRegression analysis took: ', elapsed, ' seconds\n')
	
	#initialize lists to store results
	jobsindustry = []
	capexindustry = []
	jobstotal = []
	capextotal = []
	jobsmanufacturing = []
	capexmanufacturing = []
	
	#feed results into dictionaries
	for i in range(0, len(indicators)):
		jobsindustry.append(jobsarr[i].tvalues[1])
		capexindustry.append(capexarr[i].tvalues[1])
		jobstotal.append(jobsallarr[i].tvalues[1])
		capextotal.append(capexallarr[i].tvalues[1])
		jobsmanufacturing.append(jobsmanarr[i].tvalues[1])
		capexmanufacturing.append(capexmanarr[i].tvalues[1])
	
	outputdict = {'Indicators': indicators, 'Jobs created by industry': jobsindustry, 'Capex by industry': capexindustry, 'Total jobs created': jobstotal, 'Total Capex': capextotal, 'Jobs created for manufacturing': jobsmanufacturing, 'Capex for manufacturing': capexmanufacturing}
	
	outputframe = pd.DataFrame.from_dict(outputdict)
	
	outputframe.to_csv('Output.csv')
	
	#checks if we should exit main loop
	checkIndicator = 0

	#main loop to look through results
	while checkIndicator == 0:
		#show options and ask for input
		print('These are the development indicators and their respective numbers:\n')
		#display t-stats
		for i in range(0, len(indicators)):
			#this part is for aesthetic purposes
			firstTValue = str(round(jobsarr[i].tvalues[1], 2))
			if len(firstTValue) == 3:
				firstTValue = firstTValue + '0'
			#print numbers and t-values
			print(i+1, ':\tT-values: ', firstTValue, '\t', round(capexarr[i].tvalues[1], 2), '\t', round(jobsallarr[i].tvalues[1], 2), '\t', round(capexallarr[i].tvalues[1], 2), '\t', round(jobsmanarr[i].tvalues[1], 2), '\t', round(capexmanarr[i].tvalues[1], 2), '\t', indicators[i])
		print('\n')
		
		#take input
		inputString = 'Enter a number 1-' + str(len(indicators)) + ' to choose which development indicator to view, or enter 0 to exit: '
		#make sure answer is valid
		answerIndicator = input(inputString)
		try:
			answerIndicator = int(answerIndicator)
		except ValueError:
			print('\nNot a valid input\n')
	
		#check if answer is good and then ask which results user wants to see
		if answerIndicator > 0 and answerIndicator <= len(indicators):
			checkResult = 0
			while checkResult == 0:
				print('\n')
				print('Enter a number 1-6 to choose which results to view:')
				print('1:	development indicator = b1 * (jobs created by FDI in industry) + b2 * (controls) + error term')
				print('2:	development indicator = b1 * (million invested for FDI in industry) + b2 * (controls) + error term')
				print('3:	development indicator = b1 * (jobs created by total FDI) + b2 * (controls) + error term')
				print('4:	development indicator = b1 * (millions invested for total FDI) + b2 * (controls) + error term')
				print('5:	development indicator = b1 * (jobs created by FDI for manufacturing) + b2 * (controls) + error term')
				print('6:	development indicator = b1 * (millions invested for FDI for manufacturing) + b2 * (controls) + error term')
				answerResult = input('Or enter 0 to exit: ')
				#check if result is valid
				try:
					answerResult = int(answerResult)
				except ValueError:
					print('\nNot a valid input\n')
				#check if instrumental variable is used, and display results
				if answerResult == 0:
					checkResult = 1
				elif answerResult == 1:
					print(jobsarr[answerIndicator-1].summary())
					input('Press ENTER to continue...')
				elif answerResult == 2:
					print(capexarr[answerIndicator-1].summary())
					input('Press ENTER to continue...')
				elif answerResult == 3:
					print(jobsallarr[answerIndicator-1].summary())
					input('Press ENTER to continue...')
				elif answerResult == 4:
					print(capexallarr[answerIndicator-1].summary())
					input('Press ENTER to continue...')
				elif answerResult == 5:
					print(jobsmanarr[answerIndicator-1].summary())
					input('Press ENTER to continue...')
				elif answerResult == 6:
					print(capexmanarr[answerIndicator-1].summary())
					input('Press ENTER to continue...')
		#if user chooses to exit
		elif answerIndicator == 0:
			checkIndicator = 1



Would you like to add a lag to your dependent variables?

0: No lag
1: One year lag
2: Two years lag
3: Three years lag

Select 0 - 3 to determine your lag structure: 1

Would you like to add control variables?


0 :  No additional controls
1 :  GDP per capita t-1
2 :  Trade openness t-1
3 :  Gross domestic savings (% of GDP)
4 :  War dummy
5 :  Net official development assistance and official aid received (constant 2013 US$)
6 :  External debt stocks (% of GNI)
7 :  Share of FDI in manufacturing
8 :  Share of FDI in extraction
9 :  Share of FDI in high value-added industries
10 :  Individuals using the Internet (% of population)
11 :  Arms imports (SIPRI trend indicator values)
12 :  Armed forces personnel (% of total labor force)
13 :  Chinn-Ito financial openness index

Select controls 1 - 13 to add to your regression, or select 0 to leave it be: 3

0 :  No additional controls
1 :  GDP per capita t-1
2 :  Trade openness t-1
3 :  War dummy
4 :  Net official development assistance an

/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:1118: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov_params * scale


 -0.41 	 -0.01 	 -0.32 	 0.3 	 Fertilizer consumption (kilograms per hectare of arable land)
2 :	T-values:  2.04 	 2.19 	 2.8 	 0.91 	 2.01 	 -0.87 	 Agricultural machinery, tractors
3 :	T-values:  -0.99 	 -0.13 	 -1.43 	 -0.2 	 -1.51 	 -0.33 	 Permanent cropland (% of land area)
4 :	T-values:  -0.12 	 0.24 	 -1.45 	 -1.03 	 -1.05 	 -0.92 	 Agricultural irrigated land (% of total agricultural land)
5 :	T-values:  0.43 	 -0.43 	 -1.84 	 -1.13 	 -3.58 	 -1.78 	 Agricultural machinery, tractors per 100 sq. km of arable land
6 :	T-values:  -0.2 	 -0.6 	 0.32 	 -1.01 	 0.15 	 -0.22 	 Cereal yield (kg per hectare)
7 :	T-values:  -0.78 	 -0.45 	 5.42 	 4.48 	 4.55 	 5.02 	 Access to clean fuels and technologies for cooking  (% of population)
8 :	T-values:  -1.52 	 -1.23 	 -0.28 	 -1.33 	 -0.34 	 -1.24 	 Agriculture value added per worker (constant 2010 US$)
9 :	T-values:  1.47 	 1.13 	 1.39 	 1.82 	 0.36 	 1.13 	 Access to electricity (% of population)
10 :	T-values:  -0.3 	 0.34 	 -1.09 	 0.

Enter a number 1-95 to choose which development indicator to view, or enter 0 to exit: 80


Enter a number 1-6 to choose which results to view:
1:	development indicator = b1 * (jobs created by FDI in industry) + b2 * (controls) + error term
2:	development indicator = b1 * (million invested for FDI in industry) + b2 * (controls) + error term
3:	development indicator = b1 * (jobs created by total FDI) + b2 * (controls) + error term
4:	development indicator = b1 * (millions invested for total FDI) + b2 * (controls) + error term
5:	development indicator = b1 * (jobs created by FDI for manufacturing) + b2 * (controls) + error term
6:	development indicator = b1 * (millions invested for FDI for manufacturing) + b2 * (controls) + error term
Or enter 0 to exit: 4


/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/tanaychoudhary/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


                                       OLS Regression Results                                       
Dep. Variable:     Researchers in R&D (per million people)1   R-squared:                       0.995
Model:                                                  OLS   Adj. R-squared:                  0.990
Method:                                       Least Squares   F-statistic:                     187.6
Date:                                      Sat, 16 Dec 2017   Prob (F-statistic):           3.85e-22
Time:                                              14:33:28   Log-Likelihood:                -263.78
No. Observations:                                        53   AIC:                             585.6
Df Residuals:                                            24   BIC:                             642.7
Df Model:                                                28                                         
Covariance Type:                                  nonrobust                                

Press ENTER to continue...


Enter a number 1-6 to choose which results to view:
1:	development indicator = b1 * (jobs created by FDI in industry) + b2 * (controls) + error term
2:	development indicator = b1 * (million invested for FDI in industry) + b2 * (controls) + error term
3:	development indicator = b1 * (jobs created by total FDI) + b2 * (controls) + error term
4:	development indicator = b1 * (millions invested for total FDI) + b2 * (controls) + error term
5:	development indicator = b1 * (jobs created by FDI for manufacturing) + b2 * (controls) + error term
6:	development indicator = b1 * (millions invested for FDI for manufacturing) + b2 * (controls) + error term
Or enter 0 to exit: 0
These are the development indicators and their respective numbers:

1 :	T-values:  0.33 	 -0.5 	 -0.41 	 -0.01 	 -0.32 	 0.3 	 Fertilizer consumption (kilograms per hectare of arable land)
2 :	T-values:  2.04 	 2.19 	 2.8 	 0.91 	 2.01 	 -0.87 	 Agricultural machinery, tractors
3 :	T-values:  -0.9